In [1]:
from StreamClient import StreamClient

In [2]:
trade_stream = StreamClient(["bnb","usdt"], "trade")

In [3]:
book_stream = StreamClient(["bnb","usdt"], "bookTicker")

In [6]:
#trade_stream.run()

In [7]:
# response params
# {
#   "u":400900217,     // order book updateId
#   "s":"BNBUSDT",     // symbol
#   "b":"25.35190000", // best bid price
#   "B":"31.21000000", // best bid qty
#   "a":"25.36520000", // best ask price
#   "A":"40.66000000"  // best ask qty
# }

#book_stream.run()

In [ ]:



def get_order_book(arbiPair, investmentSize):
    # Stream the entire order book using the StreamClient
    book_stream = StreamClient(["bnb","usdt"], "bookTicker")
    book_stream.run()

    # Pull the trade and append ask data to list 
    ask_list = []
    # list of lists containing best ask, qty, and trade price (ask*qty)
    # consider using a depth call to match multiple lots
    # and create trades
    # ask_list.append([float(book_stream_msg["a"]), float(book_stream_msg["A"]), ([float(book_stream_msg["a"])*float(book_stream_msg["A"]))])

    # From the trade data perform the same operation
    # on the bids
    bid_list = []
    # list of lists containing best bid, qty, and trade price (bid*qty)
    # consider using a depth call to match multiple lots
    # and create trades
    # bid_list.append([float(book_stream_msg["a"]), float(book_stream_msg["A"]), ([float(book_stream_msg["a"])*float(book_stream_msg["A"]))])


    return ask_list, bid_list


def getBTCETHParams(btc_usdLastPrice):
    eth_usdLastPrice = toFloat(driver.find_element_by_xpath(
        '//*[@id="page_content"]/div/div/div[2]/div[1]/div[1]/div[3]/div/div[3]/div/div/div/div[1]/div[2]/div[4]/span[1]').text)
    eth_btcLastPrice = toFloat(driver.find_element_by_xpath(
        '//*[@id="page_content"]/div/div/div[2]/div[1]/div[1]/div[3]/div/div[3]/div/div/div/div[3]/div[1]/div[4]/span[1]').text)

    eth_btcImpliedPrice = eth_usdLastPrice/eth_btcLastPrice
    eth_btcProfitMargin = eth_btcImpliedPrice - btc_usdLastPrice
    eth_btcProfitMarginPercent = (eth_btcProfitMargin/btc_usdLastPrice)*100
    btc_ethIsTradable = True if eth_btcProfitMarginPercent > 0.05 else False
    btc_ethProfit = initialInvestment*eth_btcProfitMargin

    tradeSizeRequest = getTradeSize('ETH-BTC', btc_ethProfit)
    askBook = tradeSizeRequest[0]
    ourBuyPosition = {'askList': btc_ethProfit,
                      'volList': btc_ethProfit/eth_btcLastPrice,
                      'buySizeList': btc_ethProfit*(btc_ethProfit/eth_btcLastPrice)
                      }
    askBook = askBook.append(ourBuyPosition, ignore_index=True)
    askBook = askBook.astype(float)
    askBook = askBook.sort_values(by=['askList'], ascending=True)
    askBook = askBook.where(
        askBook['askList'] < ourBuyPosition['askList']).dropna()
    btc_ethIsLiquid = True if btc_ethProfit / \
        eth_btcLastPrice < sum(askBook['volList']) else False

    btc_ethParamList = [btc_usdLastPrice, eth_usdLastPrice, eth_btcLastPrice, eth_btcImpliedPrice, eth_btcProfitMargin,
                        eth_btcProfitMarginPercent, btc_ethIsTradable, btc_ethIsLiquid, initialInvestment, btc_ethProfit]
    return btc_ethParamList

def get_trade_params(pair):
    # the pair should be a list of two alt coins
    # the first position should be the base
    # the second position should be the asset to
    # to arbitrage into ie. ["btc", "ltc"]
    
    # Stream the entire order book using the StreamClient
    trade_stream = StreamClient([pair[0],"usdt"], "trade")
    trade_stream.run()
    
    
    ltc_usdLastPrice = toFloat(driver.find_element_by_xpath(
        '//*[@id="page_content"]/div/div/div[2]/div[1]/div[1]/div[3]/div/div[3]/div/div/div/div[1]/div[4]/div[4]/span[1]').text)
    ltc_btcLastPrice = toFloat(driver.find_element_by_xpath(
        '//*[@id="page_content"]/div/div/div[2]/div[1]/div[1]/div[3]/div/div[3]/div/div/div/div[3]/div[3]/div[4]/span[1]').text)
    ltc_btcImpliedPrice = ltc_usdLastPrice/ltc_btcLastPrice
    ltc_btcProfitMargin = ltc_btcImpliedPrice - btc_usdLastPrice
    ltc_btcProfitMarginPercent = (ltc_btcProfitMargin/btc_usdLastPrice)*100
    ltc_btcIsTradable = True if ltc_btcProfitMarginPercent > 0.05 else False
    btc_ltcProfit = initialInvestment*ltc_btcProfitMargin
    btc_ltcParamList = [btc_usdLastPrice, ltc_usdLastPrice, ltc_btcLastPrice, ltc_btcImpliedPrice,
                        ltc_btcProfitMargin, ltc_btcProfitMarginPercent, ltc_btcIsTradable, initialInvestment, btc_ltcProfit]
    return btc_ltcParamList


while True:
    btc_usdLastPrice = getBTCUSDLastPrice()

    btc_ethParamList = getBTCETHParams(btc_usdLastPrice)

    btc_ltcParamList = getBTCLTCParams(btc_usdLastPrice)

    btc_xrpParamList = getBTCXRPParams(btc_usdLastPrice)

    btc_etcParamList = getBTCETCParams(btc_usdLastPrice)

    insertBTCETHSignal(btc_ethParamList[0], btc_ethParamList[1], btc_ethParamList[2], btc_ethParamList[3],
                       btc_ethParamList[4], btc_ethParamList[5], btc_ethParamList[6], btc_ethParamList[7], btc_ethParamList[8])
    insertBTCLTCSignal(btc_ltcParamList[0], btc_ltcParamList[1], btc_ltcParamList[2], btc_ltcParamList[3],
                       btc_ltcParamList[4], btc_ltcParamList[5], btc_ltcParamList[6], btc_ltcParamList[7], btc_ltcParamList[8])
    insertBTCXRPSignal(btc_xrpParamList[0], btc_xrpParamList[1], btc_xrpParamList[2], btc_xrpParamList[3],
                       btc_xrpParamList[4], btc_xrpParamList[5], btc_xrpParamList[6], btc_xrpParamList[7], btc_xrpParamList[8])
    insertBTCETCSignal(btc_etcParamList[0], btc_etcParamList[1], btc_etcParamList[2], btc_etcParamList[3],
                       btc_etcParamList[4], btc_etcParamList[5], btc_etcParamList[6], btc_etcParamList[7], btc_etcParamList[8])
    time.sleep(0.1 - ((time.time() - starttime) % 0.1))



